# **Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Libraries**

In [2]:
import numpy as np #perform math operation on arrays
import pandas as pd #data processing
import os as os #provide function to interact with OS
from skimage.transform import resize
from skimage.io import imread
import matplotlib.pyplot as plt
from glob import glob
import cv2
from sklearn import svm
from google.colab import files
from sklearn.model_selection import GridSearchCV

# **Load and Prepare Dataset**

In [3]:
train_path = '/content/drive/MyDrive/Preprocessed_Train_Images'
test_path = '/content/drive/MyDrive/Preprocessed_Test_Images/test/*'

In [4]:
labels = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed",
"Common wheat", "Fat Hen", "Loose Silky-bent", "Maize", "Scentless Mayweed",
"Shepherds Purse", "Small-flowered Cranesbill", "Sugar beet"]

In [5]:
output_arr=[]
img_label_arr = []
for label in labels:
  label_index = labels.index(label)
  for img in os.listdir(os.path.join(train_path, label)):
    img_path = os.path.join(train_path, label, img)
    img_array= cv2.resize(cv2.imread(img_path), (45, 45))
    flat_img = img_array.flatten()
    flat_img = np.array(flat_img)
    img_label_arr.append([flat_img, label_index])

In [6]:
x=[]
y=[]
for img, label in img_label_arr:
    x.append(img)
    y.append(label)

# **Generate Model**

In [7]:
# param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
# svc=svm.SVC(probability=True)
# model=GridSearchCV(svc,param_grid)
model = svm.SVC(C=0.1, probability=True,kernel ="linear",gamma='auto')

# **Train Model**

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [9]:
model.fit(x_train,y_train)

SVC(C=0.1, gamma='auto', kernel='linear', probability=True)

# **Evaluate Model**

In [10]:
prediction=model.predict(x_test)
accuracy = model.score(x_test,y_test)
print("Accuracy : ", accuracy)

Accuracy :  0.7558333333333334


# **Run on Test Dataset**

In [11]:
images = glob(test_path)
test_img = []
img_name = []
unflatten_img = []
for img in images:
    img_name.append(img.split('/')[-1])
    img_array= cv2.resize(cv2.imread(img), (45, 45))
    unflatten_img.append(img_array)
    flat_img = img_array.flatten()
    test_img.append(flat_img)
test_img = np.array(test_img)

In [12]:
test_prediction = model.predict(test_img)

In [13]:
pred_labels = []
for i in test_prediction:
  pred_labels.append(labels[i])
# result = {'species':pred_labels}
result = {'files':img_name,'species':pred_labels}
result = pd.DataFrame(result)
result.to_csv("SVMPrediction.csv",index=False)
files.download("SVMPrediction.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>